In [18]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
import warnings
warnings.filterwarnings("ignore")

In [274]:
ativo = 'EURUSD15'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep='\t')
df = df[df['Data'] > '2022-10-01']

#df = df.iloc[:1000]

In [26]:
ativo = 'EURUSD'
path = r'C:\Program Files\MT5 by FOREX.com Terminal\terminal64.exe'
if not mt5.initialize(path = path):
    print("initialize() failed")
    mt5.shutdown()

df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 2000))
df.columns = ['Data','Open','High','Low','Close','Vol','','']
df['Data'] = pd.to_datetime(df['Data'], unit='s').apply(lambda x: str(x))

df = df[df['Data'] > '2022-10-01']

In [27]:
df['EMA_80'] = ta.EMA(df['Close'],timeperiod=12)
df['tamanho_candle'] = abs(df['Open'] - df['Close'])
df['pos_media'] = df.apply(lambda x: 1 if x['Close'] > x['EMA_80'] else -1, axis = 1)
df['cruza_media'] = df['pos_media'].diff() #2 cruza pra cima ||| -2 cruza pra baixo || Candle que cruzou
df['minima_prox'] =  df['Low'].shift(-1)
df['maxima_prox'] =  df['High'].shift(-1)
df['fechamento_prox'] = df['Close'].shift(-1)
#verficar variações de mínimas e fechamentos
#df['entrada_min_max'] = df.apply(lambda x: 1 if )

def entrada_fechamento(x):
    if (x['cruza_media'] == 2 and x['fechamento_prox'] < x['Open']):
        return 'sell'
    elif (x['cruza_media'] == -2 and x['fechamento_prox'] > x['Open']):
        return 'call'

def entrada_min_max(x):
    if (x['cruza_media'] == 2 and x['minima_prox'] < x['Low']):
        return 'sell'
    elif (x['cruza_media'] == -2 and x['maxima_prox'] > x['High']):
        return 'call'

df['entrada_fechamento'] = df.apply(entrada_fechamento, axis = 1)
df['entrada_min_max'] = df.apply(entrada_min_max, axis = 1)
df = df.fillna(0)

metodo = 'entrada_min_max'
#excluir seguidos

lista = df[metodo].values

tempo_lista =10
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

df[metodo] = lista

#rate_stop é somado (multiplicador por tamanho de candle) em uma unidade de tamanho. rate 0.5 = stop 1.5 (somar 1)
rate_stop = 1
df['stop'] = df.apply(lambda x: x['Close'] + (x['tamanho_candle'] * rate_stop)  if x['cruza_media'] == 2 else 
x['Close'] - (x['tamanho_candle'] * rate_stop) if x['cruza_media'] == -2 else 0, axis = 1)

#rate_tp é somado em 0 unidades de tamanho. rate_tp 2 = tp 2  --Valor igual rate
rate_tp = 3.5
df['tp'] = df.apply(lambda x: x['Open'] - rate_tp * x['tamanho_candle'] if x[metodo] == 'sell' else
                                        x['Open'] + rate_tp * x['tamanho_candle'] if x[metodo] == 'call' else 0, axis =1)


for index, row in df.iterrows():
    if row['entrada_min_max'] == 'call':
        i = 1
        while True:
            try:
                if df.loc[index + i,'High'] >= row['tp']:
                    df.loc[index,'resultado_binario'] = 1
                    break
                elif df.loc[index + i,'Low'] <= row['stop']:
                    df.loc[index,'resultado_binario'] = 0
                    break
                i+=1
            except:
                df.loc[index,'resultado_binario'] = 0
                break
    elif row['entrada_min_max'] == 'sell':
        i = 1
        while True:
            try:
                if df.loc[index + i,'Low'] <= row['tp']:
                    df.loc[index,'resultado_binario'] = 1
                    break
                elif df.loc[index + i,'High'] >= row['stop']:
                    df.loc[index,'resultado_binario'] = 0
                    break
                i+=1
            except:
                df.loc[index,'resultado_binario'] = 0
                break
    

'''df['Hora'] = df['Data'].apply(lambda x: x[11:])
df['Hora_h'] = df['Hora'].apply(lambda x: x[:2])
df['Hora_h'] = df['Hora_h'].apply(lambda x: int(x))
df = df[df['Hora_h'] < 13]'''

"df['Hora'] = df['Data'].apply(lambda x: x[11:])\ndf['Hora_h'] = df['Hora'].apply(lambda x: x[:2])\ndf['Hora_h'] = df['Hora_h'].apply(lambda x: int(x))\ndf = df[df['Hora_h'] < 13]"

In [29]:
df_acao = df[(df[metodo] != 0) & (df['tamanho_candle'] > 0.0001)]
#df_acao = df[(df[metodo] != 0)]
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes

print(f'operacoes: {qtd_operacoes} --- Taxaacerto: {taxa_acerto}')

operacoes: 17 --- Taxaacerto: 0.47058823529411764


In [31]:
df_acao[['Data',metodo,'resultado_binario']].tail(10)

,Data,entrada_min_max,resultado_binario
1300,2022-12-30 12:19:00,call,1.0
1314,2022-12-30 12:33:00,call,1.0
1386,2022-12-30 13:45:00,sell,1.0
1430,2022-12-30 14:29:00,call,1.0
1463,2022-12-30 15:02:00,sell,1.0
1528,2022-12-30 16:07:00,sell,0.0
1621,2022-12-30 17:40:00,sell,0.0
1633,2022-12-30 17:52:00,sell,0.0
1682,2022-12-30 18:41:00,sell,0.0
1769,2022-12-30 20:08:00,sell,0.0


In [6]:
df.loc[45]

Data                  2022-12-30 20:08:00
Open                              1.07044
High                              1.07058
Low                               1.07044
Close                             1.07058
Vol                                   118
                                       20
                                        0
EMA_80                            1.07056
tamanho_candle                    0.00014
pos_media                               1
cruza_media                           2.0
minima_prox                        1.0704
maxima_prox                       1.07058
fechamento_prox                   1.07043
entrada_fechamento                   sell
entrada_min_max                      sell
stop                              1.07072
tp                                1.06995
resultado_binario                     0.0
Name: 45, dtype: object

In [151]:
import plotly.express as px

In [273]:
fig = px.histogram(df_acao, x="tamanho_candle", color="resultado_binario")
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=1)
fig.show()